# Mongo Agent

In [1]:
from smolagents.agents import ToolCallingAgent
from smolagents import tool, LiteLLMModel, CodeAgent
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Optional

C:\Works\AI\mongo-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL_ID = "gpt-4o-mini"
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
debug = os.getenv('DEBUG', 'False').lower() == 'true'

In [3]:
mongoclient = MongoClient(MONGODB_URI, appname="mongo-agent")
db = mongoclient.warehouse

In [4]:
@tool
def check_stock(product_id: str) -> Dict:
    """Query product stock level.

    Args:
        product_id: Product identifier

    Returns:
        Dict containing product details and quantity
    """
    return db.products.find_one({"_id": product_id})


@tool
def update_stock(product_id: str, quantity: int) -> bool:
    """Update product stock quantity.

    Args:
        product_id: Product identifier
        quantity: Amount to decrease from stock

    Returns:
        bool: Success status
    """
    result = db.products.update_one({"_id": product_id}, {"$inc": {"quantity": -quantity}})
    return result.modified_count > 0

In [5]:
@tool
def create_order(products: any, address: str) -> str:
    """Create new order for all provided products.

    Args:
        products: List of products with quantities
        address: Delivery address

    Returns:
        str: Order ID message
    """
    order = {"products": products, "status": "pending", "delivery_address": address, "created_at": datetime.now()}
    result = db.orders.insert_one(order)
    return f"Successfully ordered : {str(result.inserted_id)}"

In [6]:
from bson.objectid import ObjectId


@tool
def update_delivery_status(order_id: str, status: str) -> bool:
    """Update order delivery status to in_transit once a pending order is provided

    Args:
        order_id: Order identifier
        status: New delivery status is being set to in_transit or delivered

    Returns:
        bool: Success status
    """
    if status not in ["pending", "in_transit", "delivered", "cancelled"]:
        raise ValueError("Invalid delivery status")

    result = db.orders.update_one({"_id": ObjectId(order_id), "status": "pending"}, {"$set": {"status": status}})
    return result.modified_count > 0

In [17]:
class OrderManagementSystem:
    """Multi-agent order management system"""

    def __init__(self, model_id: str = MODEL_ID):
        self.model = LiteLLMModel(model_id=model_id, api_key=OPENAI_API_KEY)

        # Create managed agents
        self.inventory_agent = ToolCallingAgent(
            tools=[check_stock, update_stock], 
            model=self.model,
            name="inventory",
            description="Manages product inventory"
        )

        self.order_agent = ToolCallingAgent(
            tools=[create_order], 
            model=self.model,
            name="orders", 
            description="Handles order creation"
        )

        self.delivery_agent = ToolCallingAgent(
            tools=[update_delivery_status], 
            model=self.model,
            name="delivery",
            description="Manages delivery status"
        )

        # Put agents in a list for the manager
        self.managed_agents = [
            self.inventory_agent,
            self.order_agent, 
            self.delivery_agent
        ]

        # Create manager agent (remove system_prompt parameter)
        self.manager = CodeAgent(
            tools=[],
            model=self.model,
            managed_agents=self.managed_agents,
            additional_authorized_imports=["time", "json"],
        )

    def process_order(self, orders: List[Dict]) -> str:
        """Process a set of orders.

        Args:
            orders: List of orders each has address and products

        Returns:
            str: Processing result
        """
        # Include the instructions in the run() method instead
        instructions = """For each order:
        1. Create the order document
        2. Update the inventory
        3. Set delivery status to in_transit

        Use relevant agents to help you with these tasks.
        """
        
        return self.manager.run(
            f"{instructions}\n\n"
            f"Process the following {orders} as well as subtract the ordered items from inventory "
            f"to be delivered to relevant addresses"
        )